In [9]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [10]:
# Load VGG-16 with pretrained weights from ImageNet, excluding the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze convolutional base layers
for layer in base_model.layers:
    layer.trainable = False


In [11]:
# Add custom classification layers
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
output_layer = Dense(1, activation='sigmoid')(x)  # Use 'softmax' for multi-class classification

# Create the complete model
model = Model(inputs=base_model.input, outputs=output_layer)


In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [13]:
# Define data generators
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0/255)

# Load dataset
train_generator = train_datagen.flow_from_directory(
    'cat_dog/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'  # Use 'categorical' for multi-class
)

val_generator = val_datagen.flow_from_directory(
    'cat_dog/validation',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)


Found 20124 images belonging to 2 classes.
Found 4840 images belonging to 2 classes.


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)


C:\Users\singh\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
157/629 ━━━━━━━━━━━━━━━━━━━━ 22:38 3s/step - accuracy: 0.7079 - loss: 0.7641